In [1]:
# 럭키백에서 클래스마다 확률을 가지고 있다면, 럭키백확률을 구하라 
# 특징은 : 길이, 높이, 두께 + 대각선 길ㅇ, 무게
# bit.ly/fish_csv_data

import pandas as pd

fish = pd.read_csv('http://bit.ly/fish_csv_data')
# 판다스는 csv파일의 첫 줄을 자동으로 인식해 열 제목으로 만들어 줌
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
# pandas의 unique를 통해서 고유값들을 출력할 수 있음. 즉, 각 열에 따라
print(pd.unique(fish['Species']))
#print(pd.unique(fish['Length']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [3]:
# species만 target으로 두고 나머지 무게, 길이, 대각선길이, 높이, 넓이는 입력 input으로 변환
# .to_numpy()로 리스트로 변환함
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
# 확인용 5개 출력
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [4]:
# target 데이터 만들기
fish_target = fish[['Species']].to_numpy()
print(fish_target[:5])

[['Bream']
 ['Bream']
 ['Bream']
 ['Bream']
 ['Bream']]


In [5]:
#데이터셋과 테스트 셋으로 만들기
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [6]:
# StandardScaler 클래스를 이용하여 훈련세트와 테스트세트로 표준화 전처리
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)


In [7]:
# KNeighborClassifier 로 예측스코어 확인(훈련하고 난 다음에)
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


C:\Users\gunny\AppData\Local\conda\conda\envs\keras_gpu\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """


In [8]:
#정렬된 target값은 classes_ 속석에 저장 하지만 알파벳 순서로
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [9]:
#따라서 predict를 하게 되면 어떻게 될까? 
#predict는 타겟에 대한 예측값을 출력함.
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


In [10]:
print(test_target[:5])

[['Perch']
 ['Smelt']
 ['Pike']
 ['Whitefish']
 ['Perch']]


In [11]:
# 그러면 이렇게 나온 것에 대한 확률을 구하게 되면 어떨까?
import numpy as np

#predict_proba : 확률을 게산하기
#np.round(A, B) : A값에 대한 소수점 자리수 표현 4자리까지(5번째자리에서 반올림)
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))
#보는 방법은 bream, parkki, perch, pike, roach, smelt, whitefish에 대한 확률을 넣은 것 : clases_ 속성과 같음

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


In [12]:
# 4번째 샘플릐 최근접 이웃 클래스를 확인하기
distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

[[['Roach']
  ['Perch']
  ['Perch']]]


In [13]:
# sigmoid 함수 만들기
import numpy as np

import matplotlib.pyplot as plt

z = np.arange(-5, 5, 0.1)
phi = 1 / (1+np.exp(-z))
plt.plot(z, phi)
plt.xlabel('z')
plt.ylabel('phi')
plt.show()

<Figure size 640x480 with 1 Axes>

In [14]:
# 로지스틱 회귀분석 해보기
char_arr = np.array(['A', 'B', 'C', 'D', 'E'])
print(char_arr[[True, False, True, False, False]])
#위와 같이 하면 True 원소만 골라나타낼 수 있음

['A' 'C']


In [15]:
# 위와 같은 방식으로 Bream과 Smelt만 골라내기. 
bream_smelt_indexes = (train_target == 'Bream') | (train_target=='Smelt')

#형식을 변경 reshape로 해서 train_scaled에 True인것만 빼 낼수 있게 변경
a = bream_smelt_indexes.reshape(-1)

train_bream_smelt = train_scaled[a]
target_bream_smelt = train_target[a]


In [16]:
#로지스틱 회귀분석을 위한 방안 
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

C:\Users\gunny\AppData\Local\conda\conda\envs\keras_gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\gunny\AppData\Local\conda\conda\envs\keras_gpu\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [18]:
#예측 확률 출력하기
print(lr.predict_proba(train_bream_smelt[:5]))

[[0.99124329 0.00875671]
 [0.01413656 0.98586344]
 [0.98326545 0.01673455]
 [0.96045744 0.03954256]
 [0.99165598 0.00834402]]


In [19]:
# 첫번째 열과 두번째 열이 무엇인지는 clases_에 저장되어 있음
print(lr.classes_)

['Bream' 'Smelt']


In [20]:
#로지스틱회귀에서 계수 확인
print(lr.coef_, lr.intercept_)

[[-0.23113379 -0.33550888 -0.52922923 -1.3781345  -0.50236234]] [-0.828235]


In [21]:
#계수를 통한 z 값을 출력할 수 있음
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

[-4.72913982  4.24475346 -4.07340375 -3.1900321  -4.77783134]


In [22]:
#sigmoid 함수를 통해 이 값을 나타내기

from scipy.special import expit
print(expit(decisions))

[0.00875671 0.98586344 0.01673455 0.03954256 0.00834402]


In [23]:
# 7개의 생선에 대한 다중 분류
lr = LogisticRegression(C=20, max_iter=1000)

lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))


0.8991596638655462
0.925


C:\Users\gunny\AppData\Local\conda\conda\envs\keras_gpu\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\gunny\AppData\Local\conda\conda\envs\keras_gpu\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\gunny\AppData\Local\conda\conda\envs\keras_gpu\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [25]:
#처음 5개에 대한 예측 출력하기
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


In [26]:
#처음5개에 대한 예측 확률을 출력해 보기
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.    0.044 0.696 0.001 0.227 0.02  0.012]
 [0.    0.047 0.123 0.004 0.051 0.774 0.001]
 [0.    0.    0.039 0.792 0.073 0.093 0.002]
 [0.011 0.01  0.587 0.01  0.335 0.001 0.045]
 [0.    0.001 0.696 0.013 0.261 0.014 0.015]]


In [28]:
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [29]:
#다중 분류의 선형 방정식은
print(lr.coef_.shape, lr.intercept_.shape)

(7, 5) (7,)


In [30]:
# 다중분류는 sigmoid가 아닌 softmax로 분류하면 됨
# 즉, decision_function()으로 z1~z7까지 값을 구한 다음 소프트함수를 이용해서 확률로 변경하는 것
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[-10.23  -2.97   1.25  -6.42  -1.08  -3.79  -4.26]
 [-13.66  -2.88  -1.83  -5.45  -2.8    1.91  -6.55]
 [ -7.62 -12.43  -3.05   2.35  -2.38  -2.12  -5.85]
 [ -5.34  -5.35  -1.03  -5.41  -1.73  -7.33  -3.87]
 [-10.16  -6.77   1.1   -4.29  -0.94  -4.17  -4.1 ]]


In [31]:
#softmax로 변환
from scipy.special import softmax
#axis 가 1이면 각행, 즉 각 샘픙에 대해 소프트맥스로 계산
proba = softmax(decision, axis=1)
print(np.round(proba, decimals=3))

[[0.    0.013 0.89  0.    0.087 0.006 0.004]
 [0.    0.008 0.023 0.001 0.009 0.96  0.   ]
 [0.    0.    0.004 0.976 0.009 0.011 0.   ]
 [0.008 0.008 0.627 0.008 0.311 0.001 0.036]
 [0.    0.    0.873 0.004 0.113 0.004 0.005]]
